<a href="https://colab.research.google.com/github/KevinTheRainmaker/MovieRecSys/blob/main/MovieRecSys.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/data/TMDB_5000/' 

Mounted at /content/gdrive


## Dataset

TMDB 5000 Datasets from Kaggle

In [162]:
import os
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

movies = pd.read_csv(os.path.join(root_path, 'tmdb_5000_movies.csv'))
credits = pd.read_csv(os.path.join(root_path, 'tmdb_5000_credits.csv'))

In [163]:
movies.sample()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
2156,0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",NaN,14043,"[{""id"": 387, ""name"": ""california""}, {""id"": 703...",en,Nancy Drew,Intrepid teenage private eye Nancy Drew heads ...,5.0348,"[{""name"": ""Virtual Studios"", ""id"": 449}, {""nam...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-06-15,0,99.0,"[{""iso_639_1"": ""cs"", ""name"": ""\u010cesk\u00fd""...",Released,Small Town Girl. Big Time Adventure.,Nancy Drew,5.8,143


In [164]:
credits.sample()

,movie_id,title,cast,crew
3510,183894,Emma,"[{""cast_id"": 3, ""character"": ""Emma"", ""credit_i...","[{""credit_id"": ""52fe4caf9251416c7512245b"", ""de..."


### Combine datasets

In [165]:
credits.rename(columns = {'movie_id' : 'id'}, inplace = True)

In [166]:
df = movies.merge(credits, on=['title', 'id'])
df.sample()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew
2647,15000000,"[{""id"": 9648, ""name"": ""Mystery""}, {""id"": 18, ""...",http://www.midnightmeattrainthemovie.com,10185,"[{""id"": 1003, ""name"": ""photographer""}, {""id"": ...",en,The Midnight Meat Train,The photographer Leon lives with his girlfrien...,19.913961,"[{""name"": ""Lions Gate Films"", ""id"": 35}, {""nam...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2008-08-07,3533227,98.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,The most terrifying ride you'll ever take,The Midnight Meat Train,6.0,290,"[{""cast_id"": 2, ""character"": ""Leon Kaufman"", ""...","[{""credit_id"": ""52fe433d9251416c75008ca5"", ""de..."


### Simple EDA

In [167]:
df.shape

(4803, 22)

In [168]:
df['original_language'].value_counts()

en    4505
fr      70
es      32
zh      27
de      27
hi      19
ja      16
it      14
cn      12
ru      11
ko      11
pt       9
da       7
sv       5
nl       4
fa       4
th       3
he       3
ta       2
cs       2
ro       2
id       2
ar       2
vi       1
sl       1
ps       1
no       1
ky       1
hu       1
pl       1
af       1
nb       1
tr       1
is       1
xx       1
te       1
el       1
Name: original_language, dtype: int64

In [169]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4803 entries, 0 to 4802
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

In [170]:
df.isnull().sum()

budget                     0
genres                     0
homepage                3091
id                         0
keywords                   0
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
cast                       0
crew                       0
dtype: int64

### Extract some features
- genres
- id
- keywords
- title
- overview
- cast
- crew

In [197]:
extract = df[['id','title','overview','genres','keywords','cast','crew']].copy()
extract.dropna(inplace=True)
extract.sample()

,id,title,overview,genres,keywords,cast,crew
1360,45054,There Be Dragons,Arising out of the horror of the Spanish Civil...,"[{""id"": 18, ""name"": ""Drama""}]","[{""id"": 5509, ""name"": ""spanish civil war""}, {""...","[{""cast_id"": 5, ""character"": ""Josemar\u00eda"",...","[{""credit_id"": ""52fe46acc3a36847f810c013"", ""de..."


In [198]:
extract.isnull().sum()

id          0
title       0
overview    0
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [199]:
extract.duplicated().sum()

0

In [200]:
extract.shape

(4800, 7)

### Data Preprocessing

In [201]:
# genres, keywords
from ast import literal_eval
extract['genres'] = extract['genres'].apply(literal_eval)
extract['keywords'] = extract['keywords'].apply(literal_eval)
extract['genres'] = extract['genres'].apply(lambda x : [y['name'] for y in x])
extract['keywords'] = extract['keywords'].apply(lambda x : [y['name'] for y in x])

In [202]:
extract['genres'] = extract['genres'].apply(lambda x:[i.lower().replace(" ", "_") for i in x]) # ex: Science Fiction to science_fiction
extract['keywords'] = extract['keywords'].apply(lambda x:[i.replace(" ", "_") for i in x]) # ex: space war to space_war

In [203]:
extract[['genres','keywords']].sample()

,genres,keywords
2832,"[drama, horror, thriller]","[sister_sister_relationship, blindness_and_imp..."


In [204]:
# cast
extract['cast']

0       [{"cast_id": 242, "character": "Jake Sully", "...
1       [{"cast_id": 4, "character": "Captain Jack Spa...
2       [{"cast_id": 1, "character": "James Bond", "cr...
3       [{"cast_id": 2, "character": "Bruce Wayne / Ba...
4       [{"cast_id": 5, "character": "John Carter", "c...
                              ...                        
4798    [{"cast_id": 1, "character": "El Mariachi", "c...
4799    [{"cast_id": 1, "character": "Buzzy", "credit_...
4800    [{"cast_id": 8, "character": "Oliver O\u2019To...
4801    [{"cast_id": 3, "character": "Sam", "credit_id...
4802    [{"cast_id": 3, "character": "Herself", "credi...
Name: cast, Length: 4800, dtype: object

In [205]:
def convert_cast(obj):
  L = []
  counter = 0
  for i in literal_eval(obj):
    if counter != 3: # max: 3
      L.append(i['name'])
      counter+=1
    else:
      break
  return L

In [206]:
extract['cast'] = extract['cast'].apply(convert_cast)
extract['cast'] = extract['cast'].apply(lambda x:[i.replace(" ", "") for i in x])

In [207]:
extract.sample()

,id,title,overview,genres,keywords,cast,crew
190,17578,The Adventures of Tintin,"Intrepid young reporter, Tintin and his loyal ...","[adventure, animation, mystery]","[riddle, captain, treasure, liquor, treasure_h...","[JamieBell, AndySerkis, DanielCraig]","[{""credit_id"": ""52fe47359251416c750914c1"", ""de..."


In [208]:
# crew
def fetch_director(obj):
  L = []
  for i in literal_eval(obj):
    if i['job']=='Director':
      L.append(i['name'])
      break
  return L

In [209]:
# crew to director
extract['director'] = extract['crew'].apply(fetch_director)
extract = extract.drop(columns=['crew'])
extract['director'] = extract['director'].apply(lambda x:[i.replace(" ", "") for i in x])

In [210]:
extract.sample()

,id,title,overview,genres,keywords,cast,director
4716,38786,The Blood of My Brother: A Story of Death in Iraq,THE BLOOD OF MY BROTHER goes behind the scenes...,[],[],[],[]


In [211]:
# overview
extract['overview'] = extract['overview'].apply(lambda x: x.lower().split())

In [212]:
extract.sample()

,id,title,overview,genres,keywords,cast,director
157,147441,Exodus: Gods and Kings,"[the, defiant, leader, moses, rises, up, again...","[adventure, drama, action]","[moses, bible, ancient_egypt, 3d, ramses]","[ChristianBale, JoelEdgerton, JohnTurturro]",[RidleyScott]


In [213]:
extract['tags'] = extract['genres'] + extract['keywords'] + extract['cast'] + extract['director']
movies_df = extract[['id','title','overview','tags']]

In [214]:
movies_df.sample()

,id,title,overview,tags
2417,9792,The Hills Have Eyes,"[based, on, wes, craven's, 1977, suspenseful, ...","[horror, thriller, ambush, new_mexico, van, fa..."
